# Shallow Waters Equations in 1D

We will solve a shallow waters equations system in 1D:

\begin{align}

\frac{\partial h}{\partial  t}+ \frac{\partial h u}{\partial  x} = 0,\\

\frac{\partial h u}{\partial  t}
  +\frac{\partial }{\partial  x}\left(hu^2 +\frac{1}{2}gh^2\right) = 0,

\end{align}

where $x \in [-5, 5], \quad t \in [0, 3]$

and considering $g=1$.

The initial conditions:

$$
h(x, 0) = 1 + \frac{2}{5}e^{-5x^2}, \quad u(x, 0) = 0
$$

and boundary conditions:

$$
h(-5, t) = h(5, t) = 1, \quad u(-5, t) = u(5, t) = 0.
$$

The reference solution is computed by finite differences *here*.

## Implementation and Training

First, we import the libraries:

In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import datetime
import math

We set the parameter $g$ and define the $sine$ function, along with the maximum and minimum values of the domain:

In [2]:
e = torch.exp
g = 1.

x_min = -5.
x_max = 5.

t_max = 3.

Now, we set the parameters of the neural network: it has a structure with 2 inputs ($x, t$) and 2 outputs ($h(x,t), u(x,t)$), 10000 maximum training steps, 3 hidden layers with 32 neurons each, 100 samples and a target minimum loss value of $10^{-4}$.

In [3]:
inputs = 2
outputs = 2
hn_1 = 32
hn_2 = 32
hn_3 = 32
steps = 0
max_steps = 10000
loss = 10
min_loss = 1e-4
log_each = 500
samples = 100
loss_values = []

In this part, we define a new class implementing the activation function $sin(x)$, due to the oscillatory nature of the solution.

In [4]:
class Sine(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.sin(x)

The multilayer perceptron (MLP) structure is:

In [5]:
mlp = nn.Sequential(
    nn.Linear(inputs,hn_1),
    Sine(),
    nn.Linear(hn_1, hn_2),
    Sine(),
    nn.Linear(hn_2, hn_3),
    Sine(),
    nn.Linear(hn_3, outputs)
)

optimizer = torch.optim.Adam(mlp.parameters())
criterion = nn.MSELoss()
mlp.train()

Sequential(
  (0): Linear(in_features=2, out_features=32, bias=True)
  (1): Sine()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): Sine()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): Sine()
  (6): Linear(in_features=32, out_features=2, bias=True)
)

In this section, we define a function that calculates the gradients.

In [6]:
def computeGrads(y, x):
    grads, = torch.autograd.grad(y, x, grad_outputs=y.data.new(y.shape).fill_(1), create_graph=True, only_inputs=True)
    return grads

Next, we define the main training loop and the timer:

In [7]:
starttime_train = datetime.datetime.now()
print('----Training Started----')

while steps < max_steps and loss > min_loss:
    x = (x_max - x_min)*torch.rand(samples) + x_min
    t = (t_max - 0)*torch.rand(samples)
    X = torch.stack([x, t], axis=-1)
    X.requires_grad = True
    Y = mlp(X)
    h, u = Y[:, :1], Y[:, 1:] 
    grads_h = computeGrads(h, X)
    dhdx = grads_h[:, :1]
    dhdt = grads_h[:, 1:]
    grads_u = computeGrads(u, X)
    dudx = grads_u[:, :1]
    dudt = grads_u[:, 1:]
    pde_loss = criterion(dhdt, -h*dudx -u*dhdx)  + \
            criterion(h*dudt + u*dhdt, -2*h*u*dudx - (u**2)*dhdx - g*h*dhdx)

    #initial conditions
    t0 = torch.zeros(samples)
    X_initial = torch.stack([x, t0], axis=-1)
    Y_initial = mlp(X_initial)
    h, u = Y_initial[:, :1], Y_initial[:, 1:]
    h0 = (1 + 2*e(-5*(x**2))/5).unsqueeze(1)
    zero = torch.zeros(samples, 1)
    ic_loss_h = criterion(h, h0)
    ic_loss_u = criterion(u, zero)

    ic_loss = ic_loss_h + ic_loss_u

    #boundary conditions x=-5
    x_5 = -5*torch.ones(samples)
    X_boundary_5 = torch.stack([x_5, t], axis=-1)
    Y_boundary_5 = mlp(X_boundary_5)
    h_5, u_5 = Y_boundary_5[:, :1], Y_boundary_5[:, 1:]
    one =  torch.ones(samples, 1)
    bc_5_loss_h = criterion(h_5, one)
    bc_5_loss_u = criterion(u_5, zero)

    bc_5_loss = bc_5_loss_h + bc_5_loss_u
    
    #boundary conditions x=5
    x5 = 5*torch.ones(samples)
    X_boundary5 = torch.stack([x5, t], axis=-1)
    Y_boundary5 = mlp(X_boundary5)
    h5, u5 = Y_boundary5[:, :1], Y_boundary5[:, 1:]
    bc5_loss_h = criterion(h5, one)
    bc5_loss_u = criterion(u5, zero)

    bc5_loss = bc5_loss_h + bc5_loss_u

    bc_loss = bc_5_loss + bc5_loss

    optimizer.zero_grad()
    loss = pde_loss + ic_loss  + bc_loss
    loss.backward()
    optimizer.step()

    loss_values.append(loss.detach().numpy())

    if steps % log_each == 0:
        print(f'Step:{steps}| pde_loss {pde_loss.item():.5f} ic_loss {ic_loss.item():.5f} bc_loss {bc_loss.item():.5f}')

    steps+=1

endtime_train = datetime.datetime.now()
train_time = endtime_train - starttime_train
train_time_formatted = train_time.seconds + train_time.microseconds / 1e6
print('---Training Finished---')

print(f'Training Duration: {steps} steps in {train_time_formatted:.3f} seconds')

----Training Started----
Step:0| pde_loss 0.00212 ic_loss 0.82489 bc_loss 1.16497
Step:500| pde_loss 0.00063 ic_loss 0.00643 bc_loss 0.00004
Step:1000| pde_loss 0.00045 ic_loss 0.00259 bc_loss 0.00004
Step:1500| pde_loss 0.00044 ic_loss 0.00411 bc_loss 0.00003
Step:2000| pde_loss 0.00031 ic_loss 0.00166 bc_loss 0.00002
Step:2500| pde_loss 0.00062 ic_loss 0.00202 bc_loss 0.00004
Step:3000| pde_loss 0.00061 ic_loss 0.00062 bc_loss 0.00003
Step:3500| pde_loss 0.00048 ic_loss 0.00037 bc_loss 0.00003
Step:4000| pde_loss 0.00029 ic_loss 0.00025 bc_loss 0.00002
Step:4500| pde_loss 0.00018 ic_loss 0.00010 bc_loss 0.00002
Step:5000| pde_loss 0.00022 ic_loss 0.00008 bc_loss 0.00002
Step:5500| pde_loss 0.00014 ic_loss 0.00005 bc_loss 0.00002
Step:6000| pde_loss 0.00016 ic_loss 0.00006 bc_loss 0.00001
Step:6500| pde_loss 0.00021 ic_loss 0.00005 bc_loss 0.00002
Step:7000| pde_loss 0.00014 ic_loss 0.00003 bc_loss 0.00001
Step:7500| pde_loss 0.00011 ic_loss 0.00006 bc_loss 0.00003
Step:8000| pde_loss

## Visualization

To visualize the model solution, we generate the output of the trained model and display it as a GIF alongside the reference solution.

In [8]:
x = torch.linspace(x_min, x_max, samples)
t = torch.linspace(0, t_max, samples)

h = [] 
u = []
time = []

for t_ in t:
    with torch.no_grad():
        X = torch.stack([x, torch.ones(samples)*t_], axis=-1)
        Y = mlp(X) 
    h.append(Y[:, :1].detach().numpy())
    u.append(Y[:, 1:].detach().numpy())
    time.append(t_)

def update(i):
    ax.clear()
    ax.plot(x, h[i], '.-', label = '$h(x,t)$')
    ax.plot(x, h[i]*u[i], '-', label = '$hu$')
    ax.set_xlabel('$x$')
    ax.set_title(f'$t = {time[i]:.3f}$')
    ax.set_ylim(-0.3, 1.5)
    ax.grid(True)
    ax.legend()
    return ax

fig = plt.figure(dpi=100)
ax = plt.subplot(1,1,1)
anim = animation.FuncAnimation(fig, update, frames=len(h), interval=200)
# anim.save('shallow_waters.gif', dpi=300, writer=PillowWriter(fps=25))
plt.close(fig)

[Animation](https://github.com/munozmfrancisco/PINNs/raw/main/GIFs/Forward/shallow_waters.gif)